### Importing liblaries 

In [1]:
import sys

print('Python present in',sys.executable)

Python present in /Users/abhinavtiwari/miniforge3/envs/bioSimulation/bin/python


In [2]:
import numpy as np 
import json 
import logging
import os
import matplotlib.pyplot as plt
# import ssbio
# from ssbio.core.protein import Protein
# from ssbio.protein.structure.structprop import StructProp
# from ssbio.protein.structure.properties import opm
# import ssbio.databases.pdb
global_boundary = 0 ;


In [3]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

# our first handler is a console handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.ERROR)
console_handler_format = '%(asctime)s | %(levelname)s: %(message)s'
console_handler.setFormatter(logging.Formatter(console_handler_format))

# the second handler is a file handler
file_handler = logging.FileHandler('sample.log')
file_handler.setLevel(logging.INFO)
file_handler_format = '%(asctime)s | %(levelname)s | %(lineno)d: %(message)s'
file_handler.setFormatter(logging.Formatter(file_handler_format))
logger.addHandler(file_handler)

# start logging and show messages
logger.debug('Here you have some information for debugging.')
logger.info('Everything is normal. Relax!')
logger.warning('Something unexpected but not important happend.')
logger.error('Something unexpected and important happened.')
logger.critical('OMG!!! A critical error happend and the code cannot run!')

### Pre defined functions used for our analysis 

In [4]:
# This code takes in a PDF file and return list of all ATOM, Helix Record types 
def getAtomRecord(pdb_lines):
    atom_list= [];
    helix_list =[];
    sheet_list = [];
    link_list = [];
    for line in pdb_lines:
        if(line[0:4] == 'ATOM'):
            atom_list.append(line);
        if(line[0:5] == 'HELIX'):
            helix_list.append(line);
        if(line[0:5] == 'SHEET'):
            sheet_list.append(line);
        if(line[0:4] == 'LINK'):
            link_list.append(line);
    return atom_list,helix_list,sheet_list,link_list ;


In [5]:
def removeIncompleteResidues(pdb_lines):
    prev_res = '';
    count =0; 
    cnt =0 ; 
    temp_residue = [];
    pdb_lines_compelete_residue =[];
    for ind_residue in pdb_lines:
        residue_number = ind_residue[22:26];
        #print(ind_residue[13:15],residue_number, cnt)

        if(residue_number != prev_res ):
            if(count >= 4 and cnt  == 4 ):
                pdb_lines_compelete_residue = pdb_lines_compelete_residue + temp_residue;
            count = 1; 
            cnt =0 ; 
            temp_residue=[ind_residue];
            prev_res= residue_number;
        else:
            temp_residue.append(ind_residue);
            count = count + 1; 

        if(ind_residue[13:15] == 'N '):
            cnt = cnt +1 ; 
        if(ind_residue[13:15] == 'C '):
            cnt = cnt +1 ; 
        if(ind_residue[13:15] == 'O '):
            cnt = cnt +1 ; 
        if(ind_residue[13:15] == 'CA'):
            cnt = cnt +1 ; 
    return pdb_lines_compelete_residue;


In [6]:
#This takes in a list of atom records and removes all the atmos which are not present in the main chain 
def preserveMainChain(pdb_lines) :
    mainChain_lines = [];
    prev_res = '';
    count =0; 
    for ind_residue in pdb_lines:
        residue_number = ind_residue[22:26];
        if(residue_number != prev_res ):
            count = 1; 
            prev_res= residue_number;

        if(count > 4):
            continue;
        else:
            mainChain_lines.append(ind_residue);
            count = count + 1; 
    return mainChain_lines;

In [7]:
# Functino which takes in the main chain information for an alpha helix and calculates the distance bwteen n C and n+1 N atoms 
# alpha helix is a dictionary as described above which contains information about the helix 
def getHBondEuclideanDistanceHelix(alpha_helix,residue_key='mainChainHelix_residues' ):
    helix_length = int(alpha_helix['length']);
    #membrane_bound_length = int(alpha_helix['membrane_bound_length'])
    helix_residues = alpha_helix[residue_key];
    n_nplusfour_distance_array = [];
    broken_H_bonds_array= [];
    #print('Helix Lengh is ',helix_length);

    if(helix_length < 5 ):
       # print('No H bonding can occur as residue length is ', helix_length);
        alpha_helix['n_nplusfour_distances'] = [];
        alpha_helix['broken_H_bonds']= [];
    else : 
        for AcceptorResidue in helix_residues[:-4]:
            if(AcceptorResidue[13:14] == 'O'):
                AcceptorResNo = int(AcceptorResidue[23:28])
                donorPresent= 0 ; 
                for donorResidue in helix_residues:
                     DonorResNo = int(donorResidue[23:28])
                     if(DonorResNo == AcceptorResNo + 4):
                         if(donorResidue[13:14] == 'N'):
                            donorPresent =1 ; 
                            #calculate distance 
                            donor_coordinates_x =float(donorResidue[30:38]);
                            acceptor_coordinates_x = float(AcceptorResidue[30:38]);
                            donor_coordinates_y = float(donorResidue[38:46]);
                            acceptor_coordinates_y = float(AcceptorResidue[38:46]);
                            donor_coordinates_z = float(donorResidue[46:54]);
                            acceptor_coordinates_z = float(AcceptorResidue[46:54]);
                            acceptor_point = np.array((donor_coordinates_x, donor_coordinates_y, donor_coordinates_z));
                            donor_point =  np.array((acceptor_coordinates_x, acceptor_coordinates_y, acceptor_coordinates_z));
                            dist = np.linalg.norm(acceptor_point - donor_point);  # calculates the distance between the donor and acceptor atoms
                            n_nplusfour_distance_array.append(dist);
                            if(dist > 3.5 ): # H bond criteria for helix
                                broken_H_bonds_array.append(AcceptorResNo);
                            break;
                if(donorPresent == 0 ):
                    n_nplusfour_distance_array.append(0);
        alpha_helix['n_nplusfour_distances'] = n_nplusfour_distance_array;
        alpha_helix['broken_H_bonds']= broken_H_bonds_array;    


        # for res_no in range(0 , helix_length - 4):  # we loop from the N terminal to the C terminal of the helix and find distance b/w donor and acceptors 
        #     donor_atom = res_no*4 + 16;  # stores the index of nitogen atom which gives the H bond
        #     acceptor_atom = res_no*4 +3 ; # stores the index of the O aton which accepts the hydrogen bond 
        #     #print('donor atom is' , helix_residues[donor_atom]);
        #     #print('accpetor atom is' , helix_residues[acceptor_atom]);
        #     donor_coordinates_x =float(helix_residues[donor_atom][30:38]);
        #     acceptor_coordinates_x = float(helix_residues[acceptor_atom][30:38]);
        #     donor_coordinates_y = float(helix_residues[donor_atom][38:46]);
        #     acceptor_coordinates_y = float(helix_residues[acceptor_atom][38:46]);
        #     donor_coordinates_z = float(helix_residues[donor_atom][46:54]);
        #     acceptor_coordinates_z = float(helix_residues[acceptor_atom][46:54]);
        #     acceptor_point = np.array((donor_coordinates_x, donor_coordinates_y, donor_coordinates_z));
        #     donor_point =  np.array((acceptor_coordinates_x, acceptor_coordinates_y, acceptor_coordinates_z));
        #     dist = np.linalg.norm(acceptor_point - donor_point);  # calculates the distance between the donor and acceptor atoms 
        #     n_nplusfour_distance_array.append(dist);
        #     if(dist > 3.5 ): # H bond criteria for helix
                
        #         broken_H_bonds_array.append(res_no);
                
        #     #print('Distance B/w acceptor and donor is', dist); 
        # alpha_helix['n_nplusfour_distances'] = n_nplusfour_distance_array;
        # alpha_helix['broken_H_bonds']= broken_H_bonds_array;
    return alpha_helix;

In [8]:
def get_dihedral(p):
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0) 
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

In [9]:
def getCoordinateList(atom_record):
    #returns the X, Y, Z coordinates in the form of a list 
    xCoord = float(atom_record[30:38]);
    yCoord = float(atom_record[38:46]);
    zCoord = float(atom_record[46:54]);
    return [xCoord, yCoord, zCoord];

In [10]:
# takes the atom records of PDB and returns the helix present in the record 
def getAlphaHelixResidues(atom_lines, helix_records):
    logger.info('Extracting alpha helices !')
 #   print('Extracting alpha helices !');

    helix_dict_list = [];
    for ind_helix in helix_records:
        serNo = ind_helix[7:10]
        code= ind_helix[11:14];
        starting_residue =  ind_helix[21:25];

        ending_residue =  ind_helix[33:37];
        helix_length = ind_helix[71:76];
        helix_chain_identifier = ind_helix[19:20]
        len = 0 ; 
        helx_lines = [];
        flag =0 ;
        prev_res= int(starting_residue);
        for atom_line in atom_lines : 
            res_number = atom_line[22:26];
            res_chain_identifier = atom_line[21:22];
            if(res_number == starting_residue and res_chain_identifier == helix_chain_identifier):
                flag = 1;       
                len = 1; 

            if( ( int(res_number) > int(ending_residue) or (int(res_number) < int(starting_residue) ))and flag == 1 ):
                break; 

            if(flag):
                if(int(res_number) > prev_res):
                    len = len + 1; 
                    prev_res= int(res_number);
                helx_lines.append(atom_line);
    

        #create a list of dictionaries to store each alpha helix information
        helix_dict = {'starting_residue_number':starting_residue ,'ending_residue_number':ending_residue,'length':len ,
                       # 'membrane_bound_length': len, 
                      'helix_S.No':serNo, 'helix_code': code,'chain_identifier':helix_chain_identifier,'helix_residues': helx_lines} ;
        helix_dict_list.append(helix_dict);
    #print(helix_dict_list);
    return helix_dict_list;

    

In [11]:
# takes the atom records of PDB and returns the helix present in the record 
def getAlphaHelixResidues_old(atom_lines, helix_records):
    logger.info('Extracting alpha helices !')
 #   print('Extracting alpha helices !');

    helix_dict_list = [];
    for ind_helix in helix_records:
        serNo = ind_helix[7:10]
        code= ind_helix[11:14];
        strating_residue =  ind_helix[21:25];
        ending_residue =  ind_helix[33:37];
        helix_length = ind_helix[71:76];
        helx_lines = [];
        flag =0 ;
        for atom_line in atom_lines : 
            res_number = atom_line[22:26];
            
            if(res_number == strating_residue):
                flag = 1;       

            if(int(res_number) > int(ending_residue) or (int(res_number) < int(strating_residue) and flag == 1 )):
                break; 

            if(flag):
                helx_lines.append(atom_line);
    

        #create a list of dictionaries to store each alpha helix information
        helix_dict = {'strating_residue_number':strating_residue ,'ending_residue_number':ending_residue,'length':helix_length ,
                      'helix_S.No':serNo, 'helix_code': code,'helix_residues': helx_lines} ;
        helix_dict_list.append(helix_dict);
    return helix_dict_list;

    

In [12]:
def getDihedralAnglesHelix(alpha_helix):
    helix_length = int(alpha_helix['length']);
    helix_residues = alpha_helix['mainChainHelix_residues'];
    psi_angle_list = [];
    phi_angle_list = [];
    omege_angle_list = [];
    
    for res_no in range(0, helix_length-1):
        #for psi angle 
        calpha = np.array(getCoordinateList(helix_residues[(res_no* 4) + 1])); 
        first_nitogen = np.array(getCoordinateList(helix_residues[res_no*4])); 
        carbon = np.array(getCoordinateList(helix_residues[(res_no * 4) +2]));
        second_nitrogen = np.array(getCoordinateList(helix_residues[(res_no+1)*4]));
        psi_angle= get_dihedral(np.array([ first_nitogen,calpha, carbon, second_nitrogen]));
        psi_angle_list.append(psi_angle);
        second_calpha = np.array(getCoordinateList(helix_residues[((res_no+1)* 4) + 1])); 
        second_carbon = np.array(getCoordinateList(helix_residues[((res_no +1)* 4) +2]));
        phi_angle= get_dihedral(np.array([ carbon,second_nitrogen, second_calpha, second_carbon]));
        phi_angle_list.append(phi_angle);


    alpha_helix['psi_angles'] = psi_angle_list;
    alpha_helix['phi_angles'] = phi_angle_list;

    #print(alpha_helix)
    return alpha_helix;

In [13]:
def writeHelixFile(helix_dict_list,pdb_code,directory_name):
        # write the results of analysis in a JSON file
        optuput_file = '../Outputs/'+directory_name+'/' + pdb_code[:-4] + '.json';

        logger.info('Saving Json File');

        #print('Ssaving Json File')

        with open(optuput_file, "w") as output:
            output.write('[');
            for dic in helix_dict_list:
                json.dump(dic,  output, indent = 4);
                output.write(',');

            output.write('{}]');
        logger.info('Success');
        #print('Success');
        return helix_dict_list;

In [14]:
def plotRamachandranAngles(helix_dict_list, pdb_code):
    logger.info('Generating Figures') 
    x=[];
    y=[];
    for ind in helix_dict_list: 
         x= x + (ind['psi_angles']);
         y= y + (ind['phi_angles']);
        
    plt.scatter(x, y);
    plt.xlabel('psi');
    plt.ylabel('phi');
    plt.savefig('../Outputs/brokenHelix/Figures/'+pdb_code+'_ramachandran.jpg')
    plt.close()

### Analysis Codes

In [15]:
def runBrokenHelixAnalysis(pdb_lines,pdb_code):
    try:
        runHBondAnalysis= 1;
        
        logger.info('Running broken chain analysis for '+ pdb_code);

        atom_list,helix_list,sheet_list,link_list  = getAtomRecord(pdb_lines);
        print('Number of atoms records present in PDB are', len(atom_list));
        print('Number of  helix present in PDB are', len(helix_list));
        print('Number of Sheets present in PDB are', len(sheet_list));
        print('Number of links present in PDB are', len(link_list));
        # this will give us all the helix present in the protein 
        
        # Extraxt all the alpha helix from the PDB files and remove all side chain atoms 
        helix_dict_list = getAlphaHelixResidues(atom_list, helix_list);
        #print(helix_dict_list);
        logger.info('Getting The main chain residues !');
        #print('Getting The main chain residues !')

        for ind in range(0,len(helix_dict_list)):
            ind_helix = helix_dict_list[ind];
            helixResidue = ind_helix['helix_residues'];
            mainChainResidue = preserveMainChain(helixResidue);
            ind_helix['mainChainHelix_residues']= mainChainResidue;
            helix_dict_list[ind] = ind_helix;

        # for each helix verify if the H bond criteria is fullfilled 
        logger.info('Checking for H bond criteria !');
        #print('Checking for H bond criteria !');
        if(runHBondAnalysis):

            for ind in range(0,len(helix_dict_list)) : 
                alpha_helix = getHBondEuclideanDistanceHelix(helix_dict_list[ind],'mainChainHelix_residues');
                helix_dict_list[ind] = alpha_helix;
            
        logger.info('Broken Helix ran sucessfully');

        return helix_dict_list;
    except:
        logger.error('Something Bad hapenned in broken helix analysis');
        return [];
        #print('Something Bad hapenned');

In [17]:
def getDihedralAngles(helix_dict_list,pdb_code,plot =0):
    helix_dict_list_original = helix_dict_list; 
    try: 
        logger.info("Performing dihedral angles analysis for " +  pdb_code);
        for ind in range(0,len(helix_dict_list)) : 
            helix_dict_list[ind] = getDihedralAnglesHelix(helix_dict_list[ind]);
        logger.info('Dihedral angles generated sucessfully');

        try:
            if(plot):
                plotRamachandranAngles(helix_dict_list,pdb_code);
        except:
            logger.error('some error occured in plotting');
        
        return helix_dict_list;



    except:
        logger.error('Something Bad hapenned in dihedral analysis')  
        return helix_dict_list_original;    

### Code to spearate the membrane bound regions of alpha helices useing the OPM data base 

In [18]:
def getMembraneBoundRegions(pdb_lines_complete):
    z_coord_boundary = -100;
    for line in pdb_lines_complete:
        type_atom = line[0:6]
        if(type_atom == 'HETATM'):
            category = line[17:20]
            if(category == 'DUM'):
                z_coord_boundary = abs(float(line[48:55]))
                print('z_coord for boundary is- ',z_coord_boundary)
                global_boundary= z_coord_boundary;
                break
    cnt =0 ; 
    pdb_lines_membraneBound = [];
    for line in pdb_lines_complete:
        if(line[0:4] == 'ATOM'):
            [X_coord,Y_coord ,Z_coord] = getCoordinateList(line);
            if( Z_coord < z_coord_boundary and  Z_coord > - z_coord_boundary  ): # only append if it is in the transmembrane region
                pdb_lines_membraneBound.append(line) ;
                cnt = cnt + 1; 
        else:
            pdb_lines_membraneBound.append(line) ;
    print("number of membrane bound atoms are", cnt)
    return pdb_lines_membraneBound,z_coord_boundary;


    

In [19]:
def filterMembranBoundHelixRegions(helix_dict_list, pdb_code):
    try: 
        f =  open('../PDB_files_membrane_bound/'+pdb_code + '_membrane_bound.pdb',"r") 
        z_coord_boundary = float(f.readline()[:-1]);
        prev_res =0 ; 
        for ind in range(0,len(helix_dict_list)): 
            membrane_bound_length =0 ; 
            ind_helix= helix_dict_list[ind]
            membraneBoundRegion=[];
            main_chain_residues =ind_helix["mainChainHelix_residues"];
            for ind_res in main_chain_residues: 
                currr_res = int(ind_res[23:28])
                [X_coord,Y_coord,Z_coord] = getCoordinateList(ind_res);
                if(Z_coord < z_coord_boundary and  Z_coord > - z_coord_boundary):
                    membraneBoundRegion.append(ind_res);
                    if(currr_res != prev_res):
                        membrane_bound_length = membrane_bound_length +1 ; 
                        prev_res = currr_res;
            ind_helix["membraneBoundHelixRegions"] =membraneBoundRegion;
            ind_helix["membraneBoundHelixLength"] =membrane_bound_length;

            helix_dict_list[ind] = ind_helix;
            
        return helix_dict_list
    except: 
        logger.error("Error in function filterMembranBoundHelixRegions")
        return [];


In [20]:
def generateConciseReport(concise_report,parent_dir ='brokenHelix' ):
    optuput_file = '../Outputs/'+parent_dir+'/concise_report.json';

    logger.info('Saving  Concise Json File');


    with open(optuput_file, "w") as output:
        output.write('[');
        for dic in concise_report:
            json.dump(dic,  output, indent = 4);
            output.write(',');
        output.write('{}]');
        logger.info('Success');
        print('Success');


In [21]:
def runTransmembraneAnalysis(pdb_lines):
    transmembraneRegions = [];
    uniqId = 0; 
    ind =0 ; 
    while(ind < len(pdb_lines)):
        line = pdb_lines[ind];
        residues =[];
        if(line[0:4] == 'ATOM'):
            length = 1; 
            resNo = int(line[22:26])
            starting_residue = resNo;
            temp_dict = {};
            residues.append(line)
            for ind1 in range(ind+1, len(pdb_lines)):
                line1 = pdb_lines[ind1];
                resNo1 = int(line1[22:26]);
                if(line[0:4] == 'ATOM' and resNo1 == (resNo)):
                    residues.append( line1)
                else:
                    if(line[0:4] == 'ATOM' and resNo1 == (resNo + 1)):
                        resNo = resNo1;
                        length = length + 1; 
                        residues.append(line1)
                    else:
                        ending_resideue = resNo; 
                        uniqId = uniqId +1 ; 
                        ind = ind1-2; 
                        break;
            ind = ind + 1; 
            res_chain_identifier = residues[0][21:22];
            temp_dict['id'] = uniqId; 
            temp_dict['starting_residue'] = starting_residue;
            temp_dict['ending_residue'] = ending_resideue;
            temp_dict['chain_identifier'] = res_chain_identifier;
            temp_dict['length'] = length;
            residues = removeIncompleteResidues(residues)
            temp_dict['residues'] =residues ;
            temp_dict['mainChain_residues'] =preserveMainChain(residues);
            transmembraneRegions.append(temp_dict);
        ind = ind + 1; 

    return(transmembraneRegions);

In [22]:
def identifyTransmembraneBreaks(transmembraneRegionsDict,helix_dict_list):
    broken_tramsmembrane_regions = [];
    for transmembranestrand in transmembraneRegionsDict : 
        if(len(transmembranestrand) == 0):
            continue;
        if(transmembranestrand['length']> 10):
            flag =0 ; 
            #print(transmembranestrand['length']);
            residue_chain_identifier = transmembranestrand['chain_identifier'];
            transmembraneResidues = transmembranestrand['mainChain_residues'];
            prevParentHelixCode ='';
            brokenResidue=[];
            for ind_residue in transmembraneResidues:
                parentHelixCode= '';
                resNo = int(ind_residue[22:26]);
        
                for ind_helix in helix_dict_list:
                    if(len(ind_helix)==0):
                        continue;
                    helix_chain_identifier = ind_helix['chain_identifier'];
                    if(helix_chain_identifier != residue_chain_identifier):
                        continue;
                    helix_starting_residue = int(ind_helix['starting_residue_number']);
                    helix_ending_residue = int(ind_helix['ending_residue_number']);
                    if(resNo>= helix_starting_residue and resNo<= helix_ending_residue):
                        parentHelixCode = ind_helix['helix_code'];
                        break; 
                if(parentHelixCode == '' or (parentHelixCode != prevParentHelixCode and prevParentHelixCode != '') ):
                    flag = 1; 
                    brokenResidue.append(ind_residue);
                prevParentHelixCode = parentHelixCode;    
            if(flag):
                  transmembranestrand['broken_residues'] = brokenResidue; 
                  transmembranestrand['flag'] =1; 
                  del transmembranestrand['residues']
                  del transmembranestrand['mainChain_residues']
                  broken_tramsmembrane_regions.append(transmembranestrand);
        else :
            continue
    return broken_tramsmembrane_regions; 
    

### Master code to run all analysis

In [23]:
# specify PDF file and read it in text format
runBrokenHelix = 1; 
runGetDihedralAngles =1; 
getMembraneRegons =1; 
membraneAnalysis= 1; 
transmembraneAnalysis=0 ; 

concise_report = [];
concise_report_tranemembrane=[];
pdb_code_list = os.listdir('../PDB_files');
for pdb_code in pdb_code_list :
    temp_dict = {'pdb_id':pdb_code, 'helices':[]};
    if(pdb_code[0] == '.'):
        continue;
    logger.info("Performing Analysis  for " +  pdb_code)
    pdb_file = open('../PDB_files/'+pdb_code, 'r');
    pdb_lines_complete = pdb_file.readlines();
    if(getMembraneRegons):
        pdb_lines_membraneBound, z_boundary = getMembraneBoundRegions(pdb_lines_complete);
        f =  open('../PDB_files_membrane_bound/'+pdb_code + '_membrane_bound.pdb',"w+") 
        f.write(str(z_boundary)+ "\n");
        for item in pdb_lines_membraneBound:
                f.write(item)
    if(runBrokenHelix):
       helix_dict_list=  runBrokenHelixAnalysis(pdb_lines_complete,pdb_code);
       if( not runGetDihedralAngles):
        writeHelixFile(helix_dict_list, pdb_code,'brokenHelix');

    if(runGetDihedralAngles):
        helix_dict_list=getDihedralAngles(helix_dict_list,pdb_code,plot =1 );
        writeHelixFile(helix_dict_list, pdb_code,'brokenHelix');
    if(membraneAnalysis):
        try:
            helix_dict_list = filterMembranBoundHelixRegions(helix_dict_list, pdb_code);
            writeHelixFile(helix_dict_list, pdb_code,'brokenHelix');
            for ind_helix in helix_dict_list:
                if(len(ind_helix) == 0):
                    continue;
                #print(ind_helix)
                temp_helix= {'helix_code':'', 'helix_length':0, 'membrane_bound_residues':[],'membrane_bound_length':1,'broken_regions':[],'Chain_identifier':'','mainChainHelix_residues':[]}
                temp_helix['helix_code'] = ind_helix['helix_code'];
                temp_helix['helix_length'] = ind_helix['length'];
                temp_helix['starting_residue'] = ind_helix['starting_residue_number'];
                temp_helix['ending_residue'] = ind_helix['ending_residue_number'];
                temp_helix['Chain_identifier']= ind_helix['chain_identifier'];
                #temp_helix['membrane_bound_length'] = ind_helix['membraneBoundHelixLength'];
                #temp_helix['membrane_bound_residues'] = ind_helix['membraneBoundHelixRegions'];
                temp_helix['broken_regions'] = ind_helix['broken_H_bonds'];
                temp_helix['mainChainHelix_residues']= ind_helix['mainChainHelix_residues'];
                temp_dict['helices'].append(temp_helix);
            concise_report.append(temp_dict);
        except:
             logger.error("Error in finding membrane regions")
    if(transmembraneAnalysis):
        try:
            f =  open('../PDB_files_membrane_bound/'+pdb_code + '_membrane_bound.pdb',"r");
            pdb_lines_membraneBound=f.readlines();
            transmembraneRegionsDict = runTransmembraneAnalysis(pdb_lines_membraneBound); # get a dict of all transmembrane regions
            # for ind in range(0,len(transmembraneRegionsDict)) : 
            #         transmembrane_region = getHBondEuclideanDistanceHelix(transmembraneRegionsDict[ind],'mainChain_residues');
            #         transmembraneRegionsDict[ind] = transmembrane_region;
            # writeHelixFile(transmembraneRegionsDict, pdb_code, 'tranmembraneRegions') # write the transmembrane helix to directory 
        # print('../Outputs/brokenHelix/'+pdb_code[:-4] + '.json')
            f =  open('../Outputs/brokenHelix/'+pdb_code[:-4] + '.json',"r").read();
            helix_dict_list = json.loads(f)
            print(pdb_code)
            broken_tramsmembrane_regions = identifyTransmembraneBreaks(transmembraneRegionsDict,helix_dict_list);
            temp_transmembrane = {'pdb_id':pdb_code,'broken_transmembrane_regions':broken_tramsmembrane_regions };
            concise_report_tranemembrane.append(temp_transmembrane);
            writeHelixFile(broken_tramsmembrane_regions,pdb_code, 'brokenTransmembraneRegions')
        except : 
            logger.error('tranmembraneAnalysis failed')

        
generateConciseReport(concise_report,'brokenHelix');
generateConciseReport(concise_report_tranemembrane,'brokenTransmembraneRegions');

z_coord for boundary is-  15.9
number of membrane bound atoms are 1286
Number of atoms records present in PDB are 1788
Number of  helix present in PDB are 12
Number of Sheets present in PDB are 0
Number of links present in PDB are 0
z_coord for boundary is-  15.3
number of membrane bound atoms are 1214
Number of atoms records present in PDB are 3439
Number of  helix present in PDB are 22
Number of Sheets present in PDB are 12
Number of links present in PDB are 3
z_coord for boundary is-  13.9
number of membrane bound atoms are 4373
Number of atoms records present in PDB are 29050
Number of  helix present in PDB are 174
Number of Sheets present in PDB are 80
Number of links present in PDB are 19
z_coord for boundary is-  14.5
number of membrane bound atoms are 5388
Number of atoms records present in PDB are 9822
Number of  helix present in PDB are 63
Number of Sheets present in PDB are 27
Number of links present in PDB are 17
z_coord for boundary is-  15.7
number of membrane bound atoms

/var/folders/sg/ln13845560z4fpbbpyl66hjw0000gn/T/ipykernel_1883/3344171912.py:13: RuntimeWarning: invalid value encountered in true_divide
  b1 /= np.linalg.norm(b1)


z_coord for boundary is-  15.7
number of membrane bound atoms are 1551
Number of atoms records present in PDB are 3569
Number of  helix present in PDB are 21
Number of Sheets present in PDB are 3
Number of links present in PDB are 4
z_coord for boundary is-  13.9
number of membrane bound atoms are 3289
Number of atoms records present in PDB are 10220
Number of  helix present in PDB are 59
Number of Sheets present in PDB are 0
Number of links present in PDB are 0
z_coord for boundary is-  13.3
number of membrane bound atoms are 6312
Number of atoms records present in PDB are 49159
Number of  helix present in PDB are 226
Number of Sheets present in PDB are 142
Number of links present in PDB are 30
z_coord for boundary is-  15.3
number of membrane bound atoms are 1282
Number of atoms records present in PDB are 8046
Number of  helix present in PDB are 31
Number of Sheets present in PDB are 44
Number of links present in PDB are 5
z_coord for boundary is-  15.1
number of membrane bound atoms

In [ ]:
pdb_code

'5fn4.pdb'